<h1>The apartment finder:</h1>

<h2>First, we ge the apartment information from our Kijiji scraper:</h2>

In [185]:
                                                                                                                                                                                                    #Get informations from Kijiji scraper:
%run kijiji_scraper.ipynb

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/home/kurosuzume/.wdm/drivers/chromedriver/linux64/85.0.4183.87/chromedriver] found in cache


 
There are no more apartments with those filters right now. 
chrome closed


In [186]:
import html
from IPython.display import HTML


<h2>The apartment addresses are shown below.</h2>

In [187]:
text_address

['35 Patience Ln, Fredericton, NB E3B 0K4, Canada',
 '171 Priestman St, Fredericton, NB E3B 3B3, Canada',
 '504 Priestman St, Fredericton, NB E3B 3B4, Canada',
 '382 George Street, E3B 1J7, Fredericton, NB',
 '382 George Street, Fredericton E3B 1J7 NB',
 '78 York Street, E3B3N5, Fredericton, NB',
 '641 Canterbury Dr, Fredericton, NB E3B 4M6, Canada',
 '31 Corbett Ave, E3A 3X3, Fredericton, NB',
 '706 Valleyview St, Fredericton, NB E3B 4E8, Canada',
 'Valleyview St, Fredericton, NB E3B 4E8, Canada',
 '149 King St, Fredericton, NB E3B 1C8, Canada',
 '31 Waggoners Lane, Fredericton, NB, E3B 2L2',
 '527 PRIESTMAN ST, FREDERICTON, NB, E3B 3B6',
 '202 Parkside Dr, Fredericton, NB E3B 5V7, Canada',
 '171 Priestman St, Fredericton, NB E3B 3B3, Canada',
 'Greenfields Dr, Fredericton, NB E3B 5V9, Canada',
 '74 Kilburn St, Fredericton, NB E3A 1Y1, Canada',
 'E3B5L9, Canada',
 '574 Forest Hill Road, E3B 4K6, Fredericton, NB',
 '151 Queen Street, Fredericton, NB, E3B 7J2',
 '677 Windsor St, Frederi

In [188]:

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


<h2>Now, we import all libraries needed:</h2>

In [189]:
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #library to handle JSON files

!pip install pgeocode
import pgeocode

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't competed 
from geopy.geocoders import Nominatim

#Matplotlib and associated ploting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium-0.5.0 --yes
import folium

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

from geopy import distance  
print("Libraries imported successfully")



Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - folium-0.5.0

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


Libraries imported successfully


<h2>The reference point is the NBCC campus in New Brunswick.</h2>
<h3>We set the location coordinates and the radius we will use to find the apartments and amenities:</h3>

In [190]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000


The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


In [191]:
#print(len(ap_prices))
#print(len(num_bedrooms))
#print(len(num_bathrooms))
#print(len(AD_title))

<h3>Ensuring all te data is in the right format:</h3>

In [192]:
#Before finding the geo-locations of the addresses given, we need to verify if they are in the right format:
main_address = []
unaccepted_address_index = []

for address in text_address:
    house = address.split(',')
    house = house[0]+",Fredericton, CA"
    main_address.append(house)

for i in range(len(main_address)):
    if main_address is None:    
        main_address.remove(main_address[i])
        ap_prices.remove(ap_prices[i])
        title_pic_change.remove(title_pic_change[i])
        num_bedrooms.remove(num_bedrooms[i])
        num_bathrooms.remove(num_bathrooms[i])
#print (main_address)
#print(len(main_address))
#print(len(ap_prices))
#print(len(num_bedrooms))
#print(len(num_bathrooms))

In [193]:
geo_location = main_address

In [194]:
#Now, to find the geo location of each of the addresses:

index_to_remove = []
geo_location = []
for address in range(len(main_address)):
    appended_location = geolocator.geocode(main_address[address])
    geo_location.append(appended_location)
    if appended_location is None:
        index_to_remove.append(address)
index_to_remove = index_to_remove.reverse()

if index_to_remove is None:
        pass
else: 
    for i in index_to_remove:

        geo_location.remove(geo_location[address])
        ap_prices.remove(ap_prices[address])
        num_bedrooms.remove(num_bedrooms[address])
        num_bathrooms.remove(num_bathrooms[address])          

#print(len(ap_prices))
#print(len(num_bedrooms))
#print(len(num_bathrooms))
#print(len(geo_location))


<h3>Showing the Location information for all the apartments</h3>

In [195]:
geo_location

[None,
 Location(171, Priestman Street, Rabbit Town, Uptown, Fredericton, York County, New Brunswick, E3B 3H2, Canada, (45.94504295, -66.6653624, 0.0)),
 Location(Priestman Street, Rabbit Town, Uptown, Fredericton, York County, New Brunswick, E3B 3H2, Canada, (45.9445916, -66.6649631, 0.0)),
 Location(George Street, Carleton, Downtown, Fredericton, York County, New Brunswick, E3B 3L7, Canada, (45.9596959, -66.6444762, 0.0)),
 Location(George Street, Carleton, Downtown, Fredericton, York County, New Brunswick, E3B 3L7, Canada, (45.9596959, -66.6444762, 0.0)),
 Location(York Street, Rabbit Town, Downtown, Fredericton, York County, New Brunswick, E3B 0V4, Canada, (45.9546808, -66.6503261, 0.0)),
 Location(Canterbury Drive, Dunn's Crossing, Skyline Acres, Fredericton, York County, New Brunswick, E3B 4K9, Canada, (45.936282, -66.634257, 0.0)),
 Location(31, Corbett Avenue, Station Pointe, Barker's Point, Fredericton, York County, New Brunswick, E3A 6R1, Canada, (45.95220965, -66.62178275, 0

In [196]:
location_info = []
discarded_index = []
for location in range(len(geo_location)):
    if not geo_location[location] == None: 
        location_info.append([geo_location[location].address, geo_location[location].latitude, geo_location[location].longitude ])
    else: 
        discarded_index.append(location)
    

In [197]:
#print(location_info)
#print(len(location_info))

<h3>Using the Introduction_to_the_Problem notebook to retrieve the map.</h3>

In [198]:
#Get the map code from the introduction notebook:
%run Introduction_to_the_Problem.ipynb

The college address is: Duffie Drive, College Hill, Skyline Acres, Fredericton, York County, New Brunswick, E3B 9W4, Canada


In [199]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000



The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


<h3>Foursquare credentials:</h3>
<p>Please, fill with your foursquare information:</p>

In [200]:
CLIENT_ID = 'GRUEM4VGSOO2HFMKCVFHP155RMR0VF4B0G1XZF3XSD5IIUQ4' # your Foursquare ID
CLIENT_SECRET = 'QQK43IO2ZGWM2HU0SF5WH41EZA4ICAAEIEVJJHFUAACNAIED' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

<p>Since it wasn't possible to use iterative method to find the amenities, we will do it in a non pythonical way:</p>

In [201]:
#Now, to get the nearby venues according to the important amenities:
amenities_list = ["Grocery Store","Supermarket", "Gym", "Drugstore", "Hospital", "Candy Store"]
search_query0 = "Grocery Store"
search_query1 = "Supermarket"
search_query2 = "Gym"
search_query3 = "Drugstore"
search_query4 = "Hospital"
search_query5 = "Candy Store"


In [202]:
url0 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query0, radius, LIMIT)
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query1, radius, LIMIT)
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT)
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query3, radius, LIMIT)
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query4, radius, LIMIT)
url5 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query5, radius, LIMIT)


In [203]:
results0 = requests.get(url0).json()
results1 = requests.get(url1).json()
results2 = requests.get(url2).json()
results3 = requests.get(url3).json()
results4 = requests.get(url4).json()
results5 = requests.get(url5).json()



In [205]:
# assign relevant part of JSON to venues
venues0 = results0['response']['venues']
venues1 = results1['response']['venues']
venues2 = results2['response']['venues']
venues3 = results3['response']['venues']
venues4 = results4['response']['venues']
venues5 = results5['response']['venues']

# transform venues into dataframes
dataframe0 = json_normalize(venues0)
dataframe1 = json_normalize(venues1)
dataframe2 = json_normalize(venues2)
dataframe3 = json_normalize(venues3)
dataframe4 = json_normalize(venues4)
dataframe5 = json_normalize(venues5)


KeyError: 'venues'

<h2>Results from foursquare will converge into one dataframe:</h2>

In [206]:
# now converge all results into one dataframe:

dataframe0 = pd.concat([dataframe0,dataframe1], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe2], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe3], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe4], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe5], axis=0, ignore_index=True)



dataframe0

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4c545e5db426ef3b11cc7e8a,Dick's Grocery Store,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",v-1600729719,False,480 Westmorland St,45.941957,-66.663877,"[{'label': 'display', 'lat': 45.94195715102332...",1417,E3B 3M9,CA,Fredericton,NB,Canada,"[480 Westmorland St, Fredericton NB E3B 3M9, C...",NaN,NaN
1,4badfab8f964a520a3773be3,Dollar Store,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1600729719,False,NaN,45.942453,-66.650432,"[{'label': 'display', 'lat': 45.94245279861792...",376,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN
2,4c3c8e25282203bb55a3fcda,American Eagle & Aerie Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1600729719,False,1381 Regent Street,45.935129,-66.664601,"[{'label': 'display', 'lat': 45.93512887044409...",1704,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent Street, Fredericton NB E3C 1A2, C...",NaN,NaN
3,53f7c437498eacb60d222490,Great Canadian Dollar Store,"[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",v-1600729719,False,Uptown Center,45.939575,-66.663973,"[{'label': 'display', 'lat': 45.93957493230641...",1468,NaN,CA,Fredericton,NB,Canada,"[Uptown Center (Prospect St), Fredericton NB, ...",Prospect St,NaN
4,5b93ade0f193c0002c473a51,The UPS Store,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1600729719,False,527 Dundonald St,45.954230,-66.647487,"[{'label': 'display', 'lat': 45.95422952664039...",1271,E3B 1X5,CA,Fredericton,NB,Canada,"[527 Dundonald St, Fredericton NB E3B 1X5, Can...",NaN,NaN
5,4cd979481647a0936c21e74c,Everything For A Dollar Store,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1600729719,False,NaN,45.940661,-66.664114,"[{'label': 'display', 'lat': 45.94066140773827...",1453,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,NaN
6,4c1030d7ce57c928688981d2,UPS Store,"[{'id': '4bf58dd8d48988d121951735', 'name': 'P...",v-1600729719,False,527 Dundonald St,45.953991,-66.647256,"[{'label': 'display', 'lat': 45.95399123193, '...",1243,E3B 1X6,CA,Fredericton,NB,Canada,"[527 Dundonald St (Beaverbrook ct), Fredericto...",Beaverbrook ct,NaN
7,4f0a2271e4b0e8ed394e6f81,Kwik-Way Convenience Store,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",v-1600729719,False,NaN,45.955823,-66.652087,"[{'label': 'display', 'lat': 45.95582316108977...",1525,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN
8,4dbf1b1b6e810768bf9ccf5a,Raymonde's General Store,[],v-1600729719,False,NaN,45.931152,-66.639836,"[{'label': 'display', 'lat': 45.93115187969736...",1379,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,NaN
9,50672bcee4b062f08efb2b86,The Fapple Store,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E...",v-1600729719,False,NaN,45.935801,-66.664730,"[{'label': 'display', 'lat': 45.935801, 'lng':...",1676,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,NaN


In [207]:
dataframe0.shape

(271, 18)

<p>The dataframe is filtered:</p>

In [208]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe0.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe0.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Dick's Grocery Store,Smoke Shop,480 Westmorland St,45.941957,-66.663877,"[{'label': 'display', 'lat': 45.94195715102332...",1417,E3B 3M9,CA,Fredericton,NB,Canada,"[480 Westmorland St, Fredericton NB E3B 3M9, C...",NaN,4c545e5db426ef3b11cc7e8a
1,Dollar Store,Miscellaneous Shop,NaN,45.942453,-66.650432,"[{'label': 'display', 'lat': 45.94245279861792...",376,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4badfab8f964a520a3773be3
2,American Eagle & Aerie Store,Clothing Store,1381 Regent Street,45.935129,-66.664601,"[{'label': 'display', 'lat': 45.93512887044409...",1704,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent Street, Fredericton NB E3C 1A2, C...",NaN,4c3c8e25282203bb55a3fcda
3,Great Canadian Dollar Store,Discount Store,Uptown Center,45.939575,-66.663973,"[{'label': 'display', 'lat': 45.93957493230641...",1468,NaN,CA,Fredericton,NB,Canada,"[Uptown Center (Prospect St), Fredericton NB, ...",Prospect St,53f7c437498eacb60d222490
4,The UPS Store,Miscellaneous Shop,527 Dundonald St,45.954230,-66.647487,"[{'label': 'display', 'lat': 45.95422952664039...",1271,E3B 1X5,CA,Fredericton,NB,Canada,"[527 Dundonald St, Fredericton NB E3B 1X5, Can...",NaN,5b93ade0f193c0002c473a51
5,Everything For A Dollar Store,Miscellaneous Shop,NaN,45.940661,-66.664114,"[{'label': 'display', 'lat': 45.94066140773827...",1453,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,4cd979481647a0936c21e74c
6,UPS Store,Paper / Office Supplies Store,527 Dundonald St,45.953991,-66.647256,"[{'label': 'display', 'lat': 45.95399123193, '...",1243,E3B 1X6,CA,Fredericton,NB,Canada,"[527 Dundonald St (Beaverbrook ct), Fredericto...",Beaverbrook ct,4c1030d7ce57c928688981d2
7,Kwik-Way Convenience Store,Convenience Store,NaN,45.955823,-66.652087,"[{'label': 'display', 'lat': 45.95582316108977...",1525,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4f0a2271e4b0e8ed394e6f81
8,Raymonde's General Store,None,NaN,45.931152,-66.639836,"[{'label': 'display', 'lat': 45.93115187969736...",1379,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,4dbf1b1b6e810768bf9ccf5a
9,The Fapple Store,Electronics Store,NaN,45.935801,-66.664730,"[{'label': 'display', 'lat': 45.935801, 'lng':...",1676,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,50672bcee4b062f08efb2b86


In [209]:
amenities= dataframe_filtered["name"].to_list()
amenities_categories = dataframe_filtered["categories"].to_list()
amenities_list_lat = dataframe_filtered["lat"].to_list()
amenities_list_lng = dataframe_filtered["lng"].to_list()


<h3>And finally, we add the amenities to our Folium map.</h3>
<p>The amenities will be displayed as a blue dot with brown outline. </p>

In [210]:

for i in range(len(geo_location)):
    if not geo_location[i] == None: 
        tooltip = "Apartment"
        folium.Marker([geo_location[i].latitude,geo_location[i].longitude], popup=AD_title[i], tooltip = tooltip ).add_to(map_Fred)
    else:
        pass

In [211]:

# add the Italian amenities as blue circle markers with red outline
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='brown',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip = label
    ).add_to(map_Fred)
map_Fred

<h3>Now, using geopy, we can verify amenities within a 800m radius from an apartment.</h3>

In [212]:
from geopy import point
#Function to find amenities in a 500m radius from a center point:
def find_in_radius(center_name, center_lat,center_lng, amenities_list_lat, amenities_list_lng, amenities_list, amenities_categories):
    """(float, float, list) -> list
        Retruns a list of amenities within a 800 radius from a center point.
    """
    center_lat = str(center_lat)
    center_lng = str(center_lng)
    center_point = point.Point(str(center_lat)+ "," + str(center_lng))
    
    within_radius = []
    amenities_cat = []
    for amenity in range(len(amenities_list)):
        amenities_list_point = point.Point(str(amenities_list_lat[amenity])+ "," + str(amenities_list_lng[amenity]))
        amen_dis = distance.distance(center_point, amenities_list_point).km
        if amen_dis <= 0.80: #1000 meters
            within_radius.append(amenities_list[amenity])
            amenities_cat.append(amenities_categories[amenity])
    return center_name,within_radius,amenities_cat

        

In [213]:
#num_bedrooms
#num_bathrooms
#appartment_price_accepted
for type in range(len(amenities_categories)):
    if amenities_categories[type] == None:
        amenities_categories[type] = "None"

In [214]:
full_list = []
for i in range(len(location_info)):
    full_list.append(find_in_radius(location_info[i][0],location_info[i][1], location_info[i][2], amenities_list_lat, amenities_list_lng, amenities,amenities_categories))

In [215]:
print(discarded_index)
print(len(ap_prices))


[0, 16, 17, 21, 38]
42


In [216]:
reverse_discarded_index = discarded_index.reverse()

In [217]:
discarded_index


[38, 21, 17, 16, 0]

In [218]:
len(ap_prices)

42

In [219]:
for i in discarded_index:
    ap_prices.remove(ap_prices[i])
    num_bedrooms.remove(num_bedrooms[i])
    num_bathrooms.remove(num_bathrooms[i])

In [220]:
len(ap_prices)
len(num_bedrooms)

37

In [222]:
ap_prices_float = []
for i in range(len(ap_prices)):
    split = ap_prices[i].replace('$', '')
    
    ap_prices_float.append(split)
    

['450.00',
 '500.00',
 '550.00',
 '600.00',
 '650.00',
 '700.00',
 '750.00',
 '750.00',
 '750.00',
 '780.00',
 '800.00',
 '825.00',
 '825.00',
 '825.00',
 '825.00',
 '850.00',
 '859.00',
 '875.00',
 '900.00',
 '900.00',
 '900.00',
 '920.00',
 '950.00',
 '950.00',
 '975.00',
 '995.00',
 '995.00',
 '995.00',
 '1,025.00',
 '1,025.00',
 '1,035.00',
 '1,050.00',
 '1,099.00',
 '1,100.00',
 '1,150.00',
 '1,185.00',
 '1,185.00']

In [231]:
ap_prices_final = []
for i in range(len(ap_prices_float)):
    make_float = float(ap_prices_float[i].replace(',',''))
    ap_prices_final.append(make_float)
ap_prices = ap_prices_final

In [233]:
apt_address = []
amenity_name = []
amenity_type= []
for i in range(len(full_list)):
    apt_address.append(full_list[i][0])
    amenity_name.append(full_list[i][1])
    amenity_type.append(full_list[i][2])

In [234]:
separator = ","
prepared_amenity = []
for i in range(len(amenity_type)):
    sep = separator.join(amenity_type[i])
    prepared_amenity.append(sep)


In [252]:
for i in range(len(prepared_amenity)):
    prepared_amenity[i] = str(prepared_amenity[i]).split(',')
    for j in range(len(prepared_amenity[i])):
        prepared_amenity[i][j].split(",")

    

In [263]:
apartment_df= pd.DataFrame(list(zip(apt_address, amenity_name, prepared_amenity, ap_prices, num_bedrooms, num_bathrooms)))

In [264]:
len(prepared_amenity)
result = list(set(x for l in prepared_amenity for x in l))


In [265]:
apartment_df = pd.concat([apartment_df,pd.get_dummies(apartment_df[2].apply(pd.Series).stack()).sum(level=0)], axis = 1)

In [266]:
apartment_df = apartment_df.replace({5:{'Bathrooms: 1': 1, 'Bathrooms: 2': 2, "Bathrooms: 1.5": 2}})

apartment_df = apartment_df.replace({4:{'Bedrooms: Bachelor/Studio': 0, 'Bedrooms: 1': 1,'Bedrooms: 2': 2, 'Bedrooms: 1 + Den':2}})


In [267]:
apartment_df = apartment_df.rename(columns={0: "Apartment Address", 1: "Amenities",2:"Amenity type", 3:"Rent price", 4:"Bedrooms", 5:"Bathrooms"})


In [268]:
apartment_df["Bedrooms"].astype("int32").dtypes

dtype('int32')

In [269]:
apartment_df["Bathrooms"].astype("int32").dtypes

dtype('int32')

In [271]:
apartment_df

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,'Clothing Store','College Gym','College Gym'],'College Rec Center','Convenience Store','Convenience Store'],'Department Store','Department Store'],'Discount Store','Electronics Store','Furniture / Home Store','Furniture / Home Store'],'General Travel','Gym / Fitness Center','Hospital','IT Services','Miscellaneous Shop','Miscellaneous Shop'],'Mobile Phone Shop','None','None'],'Nudist Beach','Paper / Office Supplies Store','Paper / Office Supplies Store'],'Smoke Shop','Smoke Shop'],'Tourist Information Center','Veterinarian',[''],['Convenience Store',['Department Store',['Furniture / Home Store',['General Travel',['Gym / Fitness Center',['Miscellaneous Shop',['None',['Smoke Shop',['Tourist Information Center'
0,"171, Priestman Street, Rabbit Town, Uptown, Fr...","[Dick's Grocery Store, Great Canadian Dollar S...","[['Smoke Shop', 'Discount Store', 'Miscellan...",450.0,0,1,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,3,1,0,4,0,0,0,0,0,0,0,0,1,0
1,"Priestman Street, Rabbit Town, Uptown, Frederi...","[Dick's Grocery Store, Great Canadian Dollar S...","[['Smoke Shop', 'Discount Store', 'Miscellan...",500.0,0,1,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,3,1,0,4,0,0,0,0,0,0,0,0,1,0
2,"George Street, Carleton, Downtown, Fredericton...","[The UPS Store, UPS Store, Kwik-Way Convenienc...","[['Miscellaneous Shop', 'Paper / Office Suppl...",550.0,2,1,5,0,0,0,5,0,0,0,0,0,4,1,5,4,0,0,4,0,0,0,0,4,5,0,0,0,5,4,0,0,0,0,0,0,1,0,0,0
3,"George Street, Carleton, Downtown, Fredericton...","[The UPS Store, UPS Store, Kwik-Way Convenienc...","[['Miscellaneous Shop', 'Paper / Office Suppl...",600.0,0,1,5,0,0,0,5,0,0,0,0,0,4,1,5,4,0,0,4,0,0,0,0,4,5,0,0,0,5,4,0,0,0,0,0,0,1,0,0,0
4,"York Street, Rabbit Town, Downtown, Fredericto...","[The UPS Store, UPS Store, Kwik-Way Convenienc...","[['Miscellaneous Shop', 'Paper / Office Suppl...",650.0,0,1,0,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,5,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0
5,"Canterbury Drive, Dunn's Crossing, Skyline Acr...","[Raymonde's General Store, Raymonde's General ...","[['None', 'None', 'None', 'None', 'None']]",700.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,"31, Corbett Avenue, Station Pointe, Barker's P...",[],[['']],750.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,"706, Valleyview Street, College Hill, Skyline ...","[Dollar Store, Lady Beaverbrook Gym - UNB, Sou...","[['Miscellaneous Shop', 'Gym / Fitness Center...",750.0,1,1,0,8,0,4,0,0,0,0,0,0,0,0,0,4,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,"Valleyview Street, College Hill, Skyline Acres...","[Dollar Store, Lady Beaverbrook Gym - UNB, Sou...","[['Miscellaneous Shop', 'Gym / Fitness Center...",750.0,1,1,0,8,0,4,0,0,0,0,0,0,0,0,0,4,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
9,"King Street, Victoria Circle, Downtown, Freder...","[Urban Almanac, Urban Almanac, Urban Almanac, ...","[['Furniture / Home Store', 'Furniture / Home...",780.0,1,1,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [272]:
apartment_df.to_csv("apartment_dataframe.csv", index=True)
